[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

Student Number: 200149271

# DIY Covid-19 Dashboard 
-Overview of comparison among daliy new cases, the number of curried tests and the number of planed tests in the Unied Kingdom

In [33]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [35]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [36]:
## Load initial data 
filters1= [
    'areaType=overview'
]

structure1= {
    "date":"date",
    "NewCases":"newCasesByPublishDate",
    "CurriedTests":"newTestsByPublishDate",
    "PlanedTested":"plannedCapacityByPublishDate"
}
api2 = Cov19API (filters=filters1, structure=structure1)
testsdistribution=api2.get_json()
with open("testsdistribution.json", "wt") as OUTF:
    json.dump(testsdistribution, OUTF)


In [37]:
## Wrangle the data
with open("testsdistribution.json", "rt") as INFIFLE1:
    data1 =json.load(INFIFLE1)
datalist1 =data1['data']
dates1 = [dictionary ['date'] for dictionary in datalist1 ]
dates1.sort()
def parse_date1 (datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")
startdate1 =parse_date1(dates1[0])
enddate1 =parse_date1(dates1[-1])
index1 =pd.date_range(startdate1, enddate1, freq='D')
timeseriesdf1=pd.DataFrame(index=index1, columns=['NewCases','CurriedTests', 'PlanedTested'])
for entry in datalist1: 
    dates1=parse_date1(entry['date'])
    for column in ['NewCases','CurriedTests', 'PlanedTested']:
        if pd.isna(timeseriesdf1.loc[dates1, column]): 
            value1= float(entry[column]) if entry[column]!=None else 0.0
            timeseriesdf1.loc[dates1, column]=value1
timeseriesdf1.fillna(0.0, inplace=True)
timeseriesdf1.to_pickle("timeseriesdf1.pkl")



In [38]:
## Download current data
def access_api1(button):
    # Ignore the parameter, put code for polling the API here
    print("I'm downloading data from the API...")
    print("...all done.")
apibutton1=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)




NewCases:  New cases by publish date

CurriedTests: :newTestsByPublishDate

PlanedTested : planned Capacity By Publish Date

In [39]:
## Graphs and Analysis
apibutton1.on_click(access_api1)
display(apibutton1)
timeseriesdf1=pd.read_pickle("timeseriesdf1.pkl")

series1=wdg.SelectMultiple(
    options=['NewCases', 'CurriedTests', 'PlanedTested'],
    value=['NewCases', 'CurriedTests', 'PlanedTested'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale1=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls1=wdg.HBox([series1, scale1])

def timeseries1_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf1[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph1=wdg.interactive_output(timeseries1_graph, {'gcols': series1, 'gscale': scale1})

ctrls1=wdg.VBox([series1, scale1])
form1=wdg.HBox([graph1, ctrls1])
display(form1)


Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

**Data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*
